In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import os

import pandas as pd

from database.postgresql_functools import PostgreSQLManager

load_dotenv()
dir_path = Path().resolve().parent
postgres = PostgreSQLManager()

aus_weather_query = """
SELECT 
    EXTRACT(YEAR FROM date) || '-' || EXTRACT(MONTH FROM date) || '-' || EXTRACT(DAY FROM date) AS "Date",
    location AS "Location",
    min_temp AS "MinTemp",
    max_temp AS "MaxTemp",
    rainfall AS "Rainfall",
    evaporation AS "Evaporation",
    sunshine AS "Sunshine",
    wind_gust_dir AS "WindGustDir",
    wind_gust_speed AS "WindGustSpeed",
    temp_9am AS "Temp9am",
    humidity_9am AS "Humidity9am",
    cloud_9am AS "Cloud9am",
    wind_dir_9am AS "WindDir9am",
    wind_speed_9am AS "WindSpeed9am",
    pressure_9am AS "Pressure9am",
    temp_3pm AS "Temp3pm",
    humidity_3pm AS "Humidity3pm",
    cloud_3pm AS "Cloud3pm",
    wind_dir_3pm AS "WindDir3pm",
    wind_speed_3pm AS "WindSpeed3pm",
    pressure_3pm AS "Pressure3pm"
FROM 
    australian_meteorology_weather
"""

df_aus = pd.read_sql_query(aus_weather_query, postgres.engine)
df_aus['RainToday'] = df_aus['Rainfall'].apply(lambda x: 'Yes' if x >= 1 else 'No')
df_aus['RainTomorrow'] = df_aus['RainToday'].shift(-1).apply(lambda x: 'Yes' if x == 'Yes' else 'No')
csv_path = os.path.join(dir_path, 'dataCsv', 'weather_study.csv')
df_aus.to_csv(csv_path, index=False)